# Sources

**Quantum Noise** - Michael A. Nielsen 
http://michaelnielsen.org/blog/qicss/noise-web.pdf

**Modeling Language with Tensor Networks** - Tai-Danae Bradley https://www.youtube.com/watch?v=12j8OV-ptC4

# Notes

“Density matrix” = “Density operator”

The density matrix seems to be a very deep abstraction once you’ve mastered the formalism, it becomes far easier to underst and many other things, including quantum **noise**, quantum **error-correction**, quantum **entanglement**, and quantum **communication**.

## Pure states

Given a pure state $\psi$ then the probability of measurement $m$ is given by the projector $P_m$:

$$
\begin{align}
P(m|\psi) &= \langle \psi \lvert P_m \rvert \psi \rangle \\
          &= tr( | \psi \rangle \langle \psi |P_m )
\end{align}
$$

Where `tr()` in the trace of the operator. 

We define a *density matrix* for a pure state:
$$\rho \equiv | \psi \rangle \langle \psi |$$

Then the probability of outcome m becomes:
$$P(m|\psi) = tr(\rho P_m ) $$

In [112]:
using LinearAlgebra

ψ = normalize!(rand(Complex{Float64}, 4), 2)
Pm = rand(Complex{Float64}, 4, 4)

@assert ψ'*Pm*ψ ≈ tr(ψ*ψ'*Pm)

In [113]:
ρ = ψ*ψ'

@assert ψ'*Pm*ψ ≈ tr(ρ*Pm)

## Mixed states

If the state is a mixed state with probabilties $p_j$ where $\sum_{j=1..n} p_j = 1$ then:

$$ | \psi \rangle = \sum_{j=1..n} p_j | \psi_j \rangle $$

And the probability of outcome m is given by:

$$
\begin{align}
P(m|\psi) &= \sum_{j=1..n} p_j \langle \psi_j |P_m|\psi_j \rangle \\
          &= \sum_{j=1..n} p_j tr( | \psi_j \rangle \langle \psi_j |P_m )
\end{align}
$$

In [120]:
n=8
p = normalize!(rand(n), 1)
ψ = normalize!(rand(Complex{Float64}, 4, n), 2)

@assert sum(p[j]*ψ[:,j]'*Pm*ψ[:,j] for j in 1:n) ≈ sum(p[j]*tr(ψ[:,j]*ψ[:,j]'*Pm) for j in 1:n)

In [121]:
@assert sum(tr(ψ[:,j]*ψ[:,j]'*Pm) for j in 1:n) ≈ tr(ψ*ψ'*Pm)

We **generalise** the density matrix to:
$$\rho \equiv \sum_{j=1..n} p_j | \psi_j \rangle \langle \psi_j |$$

And using the [basic properties](https://en.wikipedia.org/wiki/Trace_(linear_algebra)#Basic_properties) of the trace:
$$
\begin{align}
  \operatorname{tr}(\mathbf{A}) + \operatorname{tr}(\mathbf{B}) &= \operatorname{tr}(\mathbf{A} + \mathbf{B}), \\
  c \operatorname{tr}(\mathbf{A}) &= \operatorname{tr}(c\mathbf{A})
\end{align}
$$

Then the probability of outcome m becomes:
$$P(m|\psi) = tr(\rho P_m ) $$

In [122]:
ρ = sum(p[j]*ψ[:,j]*ψ[:,j]' for j in 1:n)

@assert sum(p[j]*tr(ψ[:,j]*ψ[:,j]'*Pm) for j in 1:n) ≈ tr(ρ*Pm)

## Characterisation of $\rho$

The trace of the density matrix is:

$$
\begin{align}
tr(\rho) &= tr\left(\sum_{j=1..n} p_j | \psi_j \rangle \langle \psi_j|\right) \\
         &= \sum_{j=1..n} p_j tr\left(| \psi_j \rangle \langle \psi_j|\right) \\
         &= \sum_{j=1..n} p_j \\
         &= 1
\end{align}
$$

For any vector $|a \rangle$:

$$
\begin{align}
\langle a | \rho | a \rangle &= \langle a | \left(\sum_{j=1..n} p_j | \psi_j \rangle \langle \psi_j\right) | a \rangle \\
   &= \sum_{j=1..n} p_j \langle a | \psi_j \rangle \langle \psi_j | a \rangle \\
   &= \sum_{j=1..n} p_j |\langle a | \psi_j \rangle |^2 \\
   &\geqslant 0
\end{align}
$$

So the density matrix $\rho$ is a positive matrix and its trace is always 1.

In [172]:
ψ = normalize!(rand(Complex{Float64}, 4), 2)
ρ = ψ*ψ'
@assert tr(ρ) ≈ 1.0

a = rand(4)
@assert real(a'*ρ*a) > 0.0 && abs(imag(a'*ρ*a)) < 1e-16

## Time evolution

The time evolution of a state $| \psi \rangle$ by some operator $U$ can be written $| \psi_{'} \rangle = U | \psi \rangle$. 

Applying this to the generalised density matrix:

$$
\begin{align}
\rho^{'} &\equiv \sum_{j=1..n} p_j | \psi_j^{'} \rangle \langle \psi_j^{'} | \\
         &\equiv \sum_{j=1..n} p_j U | \psi_j \rangle \langle \psi_j | U^\dagger \\
         &\equiv U \left( \sum_{j=1..n} p_j | \psi_j \rangle \langle \psi_j | \right) U^\dagger \\
         &\equiv U \rho U^\dagger
\end{align}
$$

In [177]:
ψ = normalize!(rand(Complex{Float64}, 4), 2)
U = rand(4, 4)
ρ = ψ*ψ'

@assert (U*ψ)*(U*ψ)' ≈ U*ρ*U'

## Simple example of noise

Imagine a quantum circuit with a single X gate. With probability $p$ the X gate is applied and with probability $1-p$ the X gate fails, and nothing happens.

$$
\begin{align}
\rho^{'} &= \sum_{j=1..n} p_j | \psi_j \rangle \langle \psi_j | \\
     &= \sum_{j=1..n} p_j (pX | \psi_j \rangle \langle \psi_j | X + (1-p)| \psi_j \rangle \langle \psi_j |) \\
     &= p X \rho X + (1-p) \rho
\end{align}
$$

This is a mixed state, even if the original state was not.

## Measurement

A measurement described by the projector $P_m$ gives result m with probability $tr(\rho P_m)$ and the post measurement density matrix is:

$$
\begin{align}
\rho^{'} &\equiv \frac{P_m \rho P_m^\dagger}{tr(P_m \rho P_m^\dagger)}
\end{align}
$$

# Partial trace

In [185]:
⊗ = kron

M = rand(4,4)
N = rand(4,4)
O = M⊗N

16×16 Array{Float64,2}:
 0.352267   0.675719   0.434839   …  0.0819021  0.0527057  0.0775979
 0.649735   0.442689   0.680675      0.0536572  0.0825028  0.0657511
 0.776775   0.274345   0.524393      0.0332527  0.0635602  0.0275385
 0.552015   0.475112   0.647102      0.0575871  0.0784336  0.00886047
 0.204196   0.391689   0.25206       0.809998   0.521251   0.76743
 0.376627   0.25661    0.394562   …  0.53066    0.815939   0.650268
 0.450267   0.159028   0.303971      0.328863   0.6286     0.272352
 0.319983   0.275405   0.375101      0.569527   0.775695   0.0876286
 0.0456676  0.0875996  0.0563722     0.709524   0.456594   0.672236
 0.0842311  0.0573898  0.0882421     0.464836   0.714728   0.569607
 0.1007     0.0355659  0.0679818  …  0.28807    0.550627   0.238568
 0.0715628  0.0615932  0.0838898     0.498881   0.679476   0.0767589
 0.339281   0.650809   0.41881       0.702409   0.452015   0.665495
 0.625783   0.42637    0.655583      0.460175   0.707561   0.563895
 0.74814    0.2642

In [186]:
ρA = tr(M)*N

4×4 Array{Float64,2}:
 1.45197  2.78517  1.79232  2.63881
 2.67807  1.82467  2.8056   2.23594
 3.20171  1.1308   2.16144  0.93648
 2.2753   1.95832  2.66722  0.301311

In [187]:
ρB = M*tr(N)

4×4 Array{Float64,2}:
 1.39245   0.98072  1.28574    0.168775
 0.807151  1.7046   0.0239369  1.66916
 0.180516  1.65594  1.1949     1.46211
 1.34112   1.20361  0.848836   1.44745

In [188]:
ρA⊗ρB

16×16 Array{Float64,2}:
 2.0218    1.42398  1.86687    0.245057  …  2.58793   3.39283     0.445365
 1.17196   2.47503  0.0347558  2.42357      4.4981    0.0631649   4.40459
 0.262104  2.40437  1.73496    2.12295      4.36969   3.15311     3.85823
 1.94727   1.74761  1.23249    2.10166      3.1761    2.23991     3.81954
 3.72909   2.62644  3.44332    0.451993     2.19283   2.87485     0.377372
 2.16161   4.56504  0.0641049  4.47013   …  3.81138   0.0535216   3.73214
 0.483435  4.43472  3.20003    3.91565      3.70258   2.67173     3.2692
 3.59162   3.22337  2.27325    3.87638      2.69121   1.89795     3.23642
 4.45822   3.13998  4.11658    0.540369     0.918425  1.20407     0.158055
 2.58426   5.45762  0.0766391  5.34416      1.59632   0.0224165   1.56313
 0.57796   5.30182  3.82572    4.68126   …  1.55075   1.119       1.36924
 4.29387   3.85362  2.71773    4.63431      1.12716   0.794918    1.35551
 3.16824   2.23143  2.92545    0.384013     0.295502  0.387409    0.0508538
 1.83651  